<a href="https://colab.research.google.com/github/Tyanakai/transformer_from_scratch/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class Config:
    train_file = "date.txt"


In [ ]:
DRIVE = "/content/drive/MyDrive/portfolio/transformer_study"


## preprocess data

In [ ]:
with open(os.path.join(DRIVE, Config.train_file), mode="r") as f:
    text = f.read()

In [ ]:
text

'september 27, 1994           _1994-09-27\nAugust 19, 2003              _2003-08-19\n2/10/93                      _1993-02-10\n10/31/90                     _1990-10-31\nTUESDAY, SEPTEMBER 25, 1984  _1984-09-25\nJUN 17, 2013                 _2013-06-17\napril 3, 1996                _1996-04-03\nOctober 24, 1974             _1974-10-24\nAUGUST 11, 1986              _1986-08-11\nFebruary 16, 2015            _2015-02-16\nOctober 12, 1988             _1988-10-12\n6/3/73                       _1973-06-03\nSep 30, 1981                 _1981-09-30\nJune 19, 1977                _1977-06-19\nOCTOBER 22, 2005             _2005-10-22\nDecember 1, 2013             _2013-12-01\nWednesday, February 26, 1997 _1997-02-26\nTUESDAY, SEPTEMBER 10, 1991  _1991-09-10\n3/27/16                      _2016-03-27\nAPRIL 14, 1983               _1983-04-14\n10/2/82                      _1982-10-02\nAPR 26, 1996                 _1996-04-26\n12/21/79                     _1979-12-21\nSeptember 4, 2012            _201

In [ ]:
text_x = []
text_y = []

for line in text.split("\n")[:-1]:
    text_x.append(line[:-11].lower().lstrip())
    text_y.append(line[-10:].lstrip())

In [ ]:
text_x[-1]

'thursday, november 20, 1980  '

In [ ]:
text_y[-1]

'1980-11-20'

In [ ]:
def pad_text(text):
    # 文末の空白をpad文字で埋める
    last_char_idx = len(text.strip())
    text = text[:last_char_idx] + "＠" * (len(text) - last_char_idx)
    return text
    
def create_char_id_dict(char_list):
    # 出現文字にidを対応させる
    id_char_dict = dict()
    char_id_dict = dict()
    for id, c in enumerate(np.unique(char_list)):
        id_char_dict[id] = c
        char_id_dict[c] = id

    return id_char_dict, char_id_dict

In [ ]:
def tokenizer(text_list):
    
    # 入力文字列を文字に分解する
    char_list = []
    for text in text_list:
        text = pad_text(text) # 文末の空白をpad文字で埋める
        char_list.append(list(text)) # 文字に分解しリスト化

    # 出現文字にidを対応させる
    id_char_dict, char_id_dict = create_char_id_dict(char_list)

    # 文字をidに変換する
    token_list = []
    for text in text_list:
        token_list.append([char_id_dict[c] for c in text])

    # attention_maskを作る
    attention_mask = []
    for line in char_list:
        chars = np.array(line) # 文字リストをnp.array化
        attention_mask.append((chars != "＠") * 1) # 文字が＠ではない場所が1となる

    return np.array(token_list), np.array(attention_mask)

In [ ]:
encoder_token_list, encoder_attention_mask = tokenizer(text_x)
decoder_token_list, decoder_attention_mask = tokenizer(text_y)

## model